![Logo Pyrates LLM](../assets/pyratesllm_logo_500.png)

# **Notebook#01 : Data cleaning**

This notebook performs cleaning and filtering of data collected from the Pyrates LLM experimental study, which includes:

1. **Interaction traces** from the Pyrates application
2. **Pre-test and post-test questionnaire** from the online survey tool

## Input
- `data/raw/raw_data_interaction_traces.csv`
- `data/raw/raw_data_pre_test_BOU_STA.csv`
- `data/raw/raw_data_pre_test_LIP.csv`
- `data/raw/raw_data_pre_test_LJS.csv`
- `data/raw/raw_data_pre_test_PVA.csv`
- `data/raw/raw_data_post_test_BOU_STA.csv`
- `data/raw/raw_data_post_test_LIP.csv`
- `data/raw/raw_data_post_test_LJS.csv`
- `data/raw/raw_data_post_test_PVA.csv`


## Output (manual checking)
- `data/cleaned/cleaned_data_interaction_traces.xlsx`
- `data/cleaned/cleaned_data_pre_test.xlsx`
- `data/cleaned/cleaned_data_post_test.xlsx`
  
## Output (following notebooks analyses)
- `data/interim/interaction_data.pkl`
- `data/interim/pre_test_data.pkl`
- `data/interim/post_test_data.pkl`

## 1/ Imports

In [ ]:
# Internal
import sys
sys.path.append("../src")
import students_constants as stu_const
import interaction_constants as int_const
import tests_constants  as tes_const
import session_date_constants as ses_const

# External
import pandas as pd
import unidecode
import re

## 2/ Manual work

### 2.1/ Get raw data from interaction traces database

The data is extracted from the `JOINED_TRACES` SQL view defined as follows :

```sql
SELECT
    `trace`.`id` AS `id`,
    `trace`.`date` AS `date`,
    `trace`.`game_id` AS `game_id`,
    `trace`.`level_id` AS `level_id`,
    `trace`.`game_time` AS `game_time`,
    `trace`.`progression` AS `progression`,
    `action`.`name` AS `action_name`,
    `object`.`name` AS `object_name`,
    `trace`.`duration` AS `duration`,
    `trace`.`code` AS `code`,
    `trace`.`value` AS `value`,
    `reason`.`name` AS `reason_name`,
    `trace`.`error_message` AS `error_message`,
    `trace`.`x_pos` AS `x_pos`,
    `trace`.`y_pos` AS `y_pos`,
    `trace`.`flipped` AS `flipped`,
    `trace`.`owned_key` AS `owned_key`
FROM 
    (((`trace` LEFT JOIN `action` on(`trace`.`action_id` = `action`.`id`)) 
        LEFT JOIN `object` on(`trace`.`object_id` = `object`.`id`))
        LEFT JOIN `reason` on(`trace`.`reason_id` = `reason`.`id`))
ORDER BY `trace`.`date` DESC
```

**Actions**:
1. Filter interaction traces over the experimentation period
```sql
SELECT * FROM `JOINED_TRACES` WHERE date > '2025-10-06 00:00:00' AND date < '2025-12-19 23:59:59';
```
2. Export as `.csv` files :
- `raw_data_interaction_traces.csv`

### 2.2/ Get raw data from the online survey tool

**Actions**:
1. TODO SL ?
2. Export as `.csv` files :
- `data/raw/raw_data_pre_test_BOU_STA.csv`
- `data/raw/raw_data_pre_test_LIP.csv`
- `data/raw/raw_data_pre_test_LJS.csv`
- `data/raw/raw_data_pre_test_PVA.csv`
- `data/raw/raw_data_post_test_BOU_STA.csv`
- `data/raw/raw_data_post_test_LIP.csv`
- `data/raw/raw_data_post_test_LJS.csv`
- `data/raw/raw_data_post_test_PVA.csv`

## 3/ Data fetching

### 3.1/ Interaction traces

In [ ]:
# Load interaction traces data
interaction_data = pd.read_csv("../data/raw/raw_data_interaction_traces.csv",sep=";",header = 0, encoding="latin1")
# interaction_data.head(5)

### 3.2/ Pre and post-tests

In [ ]:
# Used to normalize pre and post-tests columns name which are complex and verbose
# (makes dataframe fusion more easy)
def normalize_column_name(col):
    # Remove accents
    col = unidecode.unidecode(col)
    # Lowercase
    col = col.lower()
    # Replace special apostrophes with '
    col = col.replace("’", "'")
    # Replace spaces by underscore
    col = col.replace(" ", "_")
    # Keep only alphanumeric + underscores
    col = re.sub(r"[^a-z0-9_]", "", col)
    # Remove duplicate underscores
    col = re.sub(r"_+", "_", col)
    # Remove leading/trailing underscores
    col = col.strip("_")
    # Remove explicit unwanted parts
    forbidden_parts = ["_p2xxr", "_p2xpxr", "_314", "_p314"]
    for bad in forbidden_parts:
        col = col.replace(bad, "")
    return col


### Load pre-test datasets for each school

In [ ]:
pre_test_data_BOU_STA = pd.read_csv("../data/raw/raw_data_pre_test_BOU_STA.csv",sep="\t",header = 0, encoding="latin1")
pre_test_data_BOU_STA.columns = [normalize_column_name(c) for c in pre_test_data_BOU_STA.columns]
# pre_test_data_BOU_STA.head(5)

In [ ]:
pre_test_data_LJS = pd.read_csv("../data/raw/raw_data_pre_test_LJS.csv",sep=",",header = 0, encoding="utf-8")
pre_test_data_LJS.columns = [normalize_column_name(c) for c in pre_test_data_LJS.columns]
# pre_test_data_LJS.head(5)

In [ ]:
pre_test_data_PVA = pd.read_csv("../data/raw/raw_data_pre_test_PVA.csv",sep="\t",header = 0, encoding="latin1")
pre_test_data_PVA.columns = [normalize_column_name(c) for c in pre_test_data_PVA.columns]
# pre_test_data_PVA.head(5)

In [ ]:
pre_test_data_LIP = pd.read_csv("../data/raw/raw_data_pre_test_LIP.csv",sep="\t",header = 0, encoding="latin1")
pre_test_data_LIP.columns = [normalize_column_name(c) for c in pre_test_data_PVA.columns]
# pre_test_data_LIP.head(5)

### Concatenate datasets

In [ ]:
pre_test_data = pd.concat(
    [pre_test_data_BOU_STA, pre_test_data_LJS, pre_test_data_PVA,pre_test_data_LIP],
    ignore_index=True
)
# pre_test_data.head(5)

### Load post-test datasets for each school

In [ ]:
post_test_data_BOU_STA = pd.read_csv("../data/raw/raw_data_post_test_BOU_STA.csv",sep="\t",header = 0, encoding="latin1")
post_test_data_BOU_STA.columns = [normalize_column_name(c) for c in post_test_data_BOU_STA.columns]
# post_test_data_BOU_STA.head(5)


In [ ]:
post_test_data_LJS = pd.read_csv("../data/raw/raw_data_post_test_LJS.csv",sep=",",header = 0, encoding="utf-8")
post_test_data_LJS.columns = [normalize_column_name(c) for c in post_test_data_LJS.columns]
# post_test_data_LJS.head(5)

In [ ]:
post_test_data_PVA = pd.read_csv("../data/raw/raw_data_post_test_PVA.csv",sep="\t",header = 0, encoding="latin1")
post_test_data_PVA.columns = [normalize_column_name(c) for c in post_test_data_PVA.columns]
# post_test_data_PVA.head(5)

In [ ]:
post_test_data_LIP = pd.read_csv("../data/raw/raw_data_post_test_LIP.csv",sep="\t",header = 0, encoding="latin1")
post_test_data_LIP.columns = [normalize_column_name(c) for c in post_test_data_PVA.columns]
# post_test_data_LIP.head(5)

### Concatenate datasets

In [ ]:
post_test_data = pd.concat(
    [post_test_data_BOU_STA, post_test_data_LJS, post_test_data_PVA,post_test_data_LIP],
    ignore_index=True
)
# post_test_data.head(5)

## 4/ Data cleaning

### 4.2/ Interaction traces

### Filter and reorder columns

See `src/interaction_constants.py` for more details

In [ ]:
interaction_data=interaction_data[int_const.INTERACTION_DATA_KEYS]

### Set data to the right type

In [ ]:
# date : object > datetime
interaction_data[int_const.DATE_DATA_KEY] = pd.to_datetime(interaction_data[int_const.DATE_DATA_KEY], format="%Y-%m-%d %H:%M:%S.%f")
# duration : float > Int64 (nullable)
interaction_data[int_const.DURATION_DATA_KEY] = interaction_data[int_const.DURATION_DATA_KEY].astype("Int64")
# value : float > Int64 (nullable)
interaction_data[int_const.VALUE_DATA_KEY] = interaction_data[int_const.VALUE_DATA_KEY].astype("Int64")
# x_pos : float > Int64 (nullable)
interaction_data[int_const.X_POS_DATA_KEY] = interaction_data[int_const.X_POS_DATA_KEY].astype("Int64")
# y_pos : float > Int64 (nullable)
interaction_data[int_const.Y_POS_DATA_KEY] = interaction_data[int_const.Y_POS_DATA_KEY].astype("Int64")
# flipped : float > boolean (nullable)
interaction_data[int_const.FLIPPED_DATA_KEY] = interaction_data[int_const.Y_POS_DATA_KEY].astype("boolean")
# owned_key : float > boolean (nullable)
interaction_data[int_const.OWNED_KEY_DATA_KEY] = interaction_data[int_const.OWNED_KEY_DATA_KEY].astype("boolean")

In [ ]:
print(interaction_data.info())

### Remove unsupported characters

In [ ]:
def remove_illegal_char(s):
    if isinstance(s, str):
        return s.replace('\x19', '')
    return s

interaction_data[int_const.CODE_DATA_KEY] = interaction_data[int_const.CODE_DATA_KEY].apply(remove_illegal_char)

### Timestamp changes due to experimental problem

Chronological anomalies are present in the traces due to an experimental incident during session BOU_2_5 Session 1: computer's internal clock was set back by approximately 7 minutes (472 seconds) for 15 students, impacting the trace data (see details below).

In [ ]:
PROBLEMATIC_GAMES = [
    { "game_id" : "PnMK39E","last_correct_id" : "16420"},
    { "game_id" : "1y9zZYa","last_correct_id" : "16720"},
    { "game_id" : "CCbUYFz","last_correct_id" : "16379"},
    { "game_id" : "Arxwr9G","last_correct_id" : "16193"},
    { "game_id" : "guH7Ye8","last_correct_id" : "16997"},
    { "game_id" : "W7Rp7uy","last_correct_id" : "17108"},
    { "game_id" : "fu46Vr2","last_correct_id" : "17993"},
    { "game_id" : "FPJCjd3","last_correct_id" : "16809"},
    { "game_id" : "AyVKNE4","last_correct_id" : "17639"},
    { "game_id" : "p6UCEdQ","last_correct_id" : "15901"},
    { "game_id" : "SzebmX3","last_correct_id" : "17463"},
    { "game_id" : "7Kq3hhp","last_correct_id" : "16654"},
    { "game_id" : "CmQ3bi3","last_correct_id" : "16131"},
    { "game_id" : "rVUtbKs","last_correct_id" : "16142"},
    { "game_id" : "PuDNWgY","last_correct_id" : "16063"},
]
TIME_DELTA = 472

In [ ]:
# Correct timestamps for problematic games:
# - For each game_id in PROBLEMATIC_GAMES
#    - For trace ids < last_correct_id: subtract 472 seconds from date
total_corrected_traces = 0
for game_info in PROBLEMATIC_GAMES:
    game_id = game_info["game_id"]
    last_correct_id = int(game_info["last_correct_id"])
    # Find traces for this game_id with id < last_correct_id
    mask = (
        (interaction_data[int_const.GAME_ID_DATA_KEY] == game_id) & 
        (interaction_data[int_const.ID_DATA_KEY] < last_correct_id)
    )
    traces_to_correct = interaction_data[mask]
    if len(traces_to_correct) > 0:
        # Subtract TIME_DELTA seconds
        interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
           interaction_data.loc[mask, int_const.DATE_DATA_KEY] - 
            pd.to_timedelta(TIME_DELTA, unit='s')
        )
        total_corrected_traces +=len(traces_to_correct)
    else:
        print(f"No traces to correct")
print(f"Total corrected traces : {total_corrected_traces}")

For game_id "CVtd56h", traces before id 66160 are late by one hour

In [ ]:
mask = (interaction_data[int_const.GAME_ID_DATA_KEY] == "CVtd56h") & \
       (interaction_data[int_const.ID_DATA_KEY] <= 66160)

interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
    pd.to_datetime(interaction_data.loc[mask, int_const.DATE_DATA_KEY]) 
    + pd.Timedelta(hours=1)
)

For game_id "J3Ua1SU", traces before id 60657 are head by one hour


In [ ]:
mask = (interaction_data[int_const.GAME_ID_DATA_KEY] == "J3Ua1SU") & \
       (interaction_data[int_const.ID_DATA_KEY] <= 60657)

interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
    pd.to_datetime(interaction_data.loc[mask, int_const.DATE_DATA_KEY]) 
    - pd.Timedelta(hours=1)
)

For game_id "Pu9T7qe", traces before id 68779 are late by one hour

In [ ]:
mask = (interaction_data[int_const.GAME_ID_DATA_KEY] == "Pu9T7qe") & \
       (interaction_data[int_const.ID_DATA_KEY] <= 68779)

interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
    pd.to_datetime(interaction_data.loc[mask, int_const.DATE_DATA_KEY]) 
    + pd.Timedelta(hours=1)
)

For game_id "e5wqMLP", traces before id 68644 are late by one hour

In [ ]:
mask = (interaction_data[int_const.GAME_ID_DATA_KEY] == "e5wqMLP") & \
       (interaction_data[int_const.ID_DATA_KEY] <= 68644)

interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
    pd.to_datetime(interaction_data.loc[mask, int_const.DATE_DATA_KEY]) 
    + pd.Timedelta(hours=1)
)

For game_id "k18if8F", traces before id 66402 are late by one hour

In [ ]:
mask = (interaction_data[int_const.GAME_ID_DATA_KEY] == "k18if8F") & \
       (interaction_data[int_const.ID_DATA_KEY] <= 66402)

interaction_data.loc[mask, int_const.DATE_DATA_KEY] = (
    pd.to_datetime(interaction_data.loc[mask, int_const.DATE_DATA_KEY]) 
    + pd.Timedelta(hours=1)
)

### 4.3/ Pre and post-test

### Removal of technical columns

In [ ]:
cols_to_drop_pre_test = [
    0,  # ID de la réponse
    2,  # Dernière page
    3,  # Langue de départ
    4,  # Tête de série
    6,  # Date de la dernière action
    7,  # Quel est le code de ton groupe ?
    18, # Temps total
    19, # Durée pour le groupe : Information
    20, # Durée pour la question: I1
    21, # Durée pour la question: I2
    22, # Durée pour le groupe : Test de connaissances en Scratch
    23, # Durée pour la question: Q1
    24, # Durée pour la question: Q2
    25, # Durée pour la question: Q3
    26, # Durée pour la question: Q4
    27, # Durée pour la question: Q5
    28, # Durée pour la question: Q6
    29, # Durée pour la question: Q7
    30, # Durée pour la question: Q8
    31, # Durée pour la question: Q9
]
pre_test_data = pre_test_data.drop(pre_test_data.columns[cols_to_drop_pre_test], axis=1)
# pre_test_data.head(5)

In [ ]:
cols_to_drop_post_test = [
    0,  # ID de la réponse
    2,  # Dernière page
    3,  # Langue de départ
    4,  # Tête de série
    6,  # Date de la dernière action
    7,  # Quel est le code de ton groupe ?
    29, # Temps total
    30, # Durée pour le groupe : Informations
    31, # Durée pour la question: I1
    32, # Durée pour la question: I2
    33, # Durée pour le groupe : Test de connaissances en Python
    34, # Durée pour la question: Q1
    35, # Durée pour la question: Q2
    36, # Durée pour la question: Q3
    37, # Durée pour la question: Q4
    38, # Durée pour la question: Q5
    39, # Durée pour la question: Q6
    40, # Durée pour la question: Q7
    41, # Durée pour la question: Q8
    42, # Durée pour la question: Q9
    43, # Durée pour le groupe : Avis sur les aides du perroquet assistant numérique 
    44, # Durée pour la question: A1
    45, # Durée pour la question: A2
    46, # Durée pour la question: A9
    47, # Durée pour la question: A4
    48, # Durée pour la question: A3
    49, # Durée pour la question: G03Q17
    50, # Durée pour la question: G03Q18
    51, # Durée pour le groupe : Avis sur le jeu
    52, # Durée pour la question: G04Q19
    53, # Durée pour la question: G04Q20
    54, # Durée pour la question: G04Q21

]
post_test_data = post_test_data.drop(post_test_data.columns[cols_to_drop_post_test], axis=1)
# post_test_data.head(5)

### Rename and reorder revelant columns

See `src/tests_constants.py` for more details

In [ ]:
# Rename
pre_test_data.columns = [
    tes_const.T2_KEY, tes_const.T1_KEY,
    tes_const.STUDENT_ID_KEY,
    tes_const.Q1_KEY,tes_const.Q2_KEY, tes_const.Q3_KEY, tes_const.Q4_KEY, tes_const.Q5_KEY, tes_const.Q6_KEY, tes_const.Q7_KEY, tes_const.Q8_KEY, tes_const.Q9_KEY,
]
# Reorder
pre_test_data = pre_test_data[tes_const.PRE_TEST_KEYS]
# pre_test_data.head(5)

In [ ]:
# Rename
post_test_data.columns = [
    tes_const.T2_KEY, tes_const.T1_KEY,
    tes_const.STUDENT_ID_KEY,
    tes_const.Q1_KEY, tes_const.Q2_KEY, tes_const.Q3_KEY, tes_const.Q4_KEY, tes_const.Q5_KEY, tes_const.Q6_KEY, tes_const.Q7_KEY, tes_const.Q8_KEY, tes_const.Q9_KEY,
    tes_const.QA_KEY, tes_const.QB_KEY, tes_const.QC_KEY, tes_const.QD_KEY, tes_const.QE_KEY, tes_const.QF_KEY, tes_const.QG_KEY, 
    tes_const.QH_KEY, 
    tes_const.QI_KEY, tes_const.QJ_KEY, 
    tes_const.QK_KEY, 
]
# Reorder
post_test_data = post_test_data[tes_const.POST_TEST_KEYS]
# post_test_data.head(5)

### Set data to the right type

In [ ]:
#  _student_id : float > int
pre_test_data[tes_const.STUDENT_ID_KEY] = pre_test_data[tes_const.STUDENT_ID_KEY].astype(int)
# _T1 : object > datetime
pre_test_data[tes_const.T1_KEY] = pd.to_datetime(pre_test_data[tes_const.T1_KEY], format="%d/%m/%Y %H:%M")
# _T2 : object > datetime
pre_test_data[tes_const.T2_KEY] = pd.to_datetime(pre_test_data[tes_const.T2_KEY], format="%d/%m/%Y %H:%M")

In [ ]:
print(pre_test_data.info())

In [ ]:
#  _student_id : float > int
post_test_data[tes_const.STUDENT_ID_KEY] = post_test_data[tes_const.STUDENT_ID_KEY].astype(int)
#  _QA : float > Int64 (nullable)
post_test_data[tes_const.QA_KEY] = post_test_data[tes_const.QA_KEY].astype("Int64")
#  _QB : float > Int64 (nullable)
post_test_data[tes_const.QB_KEY] = post_test_data[tes_const.QB_KEY].astype("Int64")
#  _QC : float > Int64 (nullable)
post_test_data[tes_const.QC_KEY] = post_test_data[tes_const.QC_KEY].astype("Int64")
#  _QD : float > Int64 (nullable)
post_test_data[tes_const.QD_KEY] = post_test_data[tes_const.QD_KEY].astype("Int64")
#  _QE : float > Int64 (nullable)
post_test_data[tes_const.QE_KEY] = post_test_data[tes_const.QE_KEY].astype("Int64")
#  _QI : float > Int64 (nullable)
post_test_data[tes_const.QI_KEY] = post_test_data[tes_const.QI_KEY].astype("Int64")
#  _QJ : float > Int64 (nullable)
post_test_data[tes_const.QJ_KEY] = post_test_data[tes_const.QJ_KEY].astype("Int64")
# _T1 : object > datetime
post_test_data[tes_const.T1_KEY] = pd.to_datetime(post_test_data[tes_const.T1_KEY], format="%d/%m/%Y %H:%M")
# _T2 : object > datetime
post_test_data[tes_const.T2_KEY] = pd.to_datetime(post_test_data[tes_const.T2_KEY], format="%d/%m/%Y %H:%M")

In [ ]:
print(post_test_data.info())

### Correct Q4 columns values (bug in the form)

For the values in column Q4: adding "A)", "B)", "C)", etc. before the answer text


In [ ]:
ANS_MAPPING = {
    "c"  : "A) c",
    "2"  : "B) 2",
    "9"  : "C) 9",
    "12" : "D) 12",
    "14" : "E) 14",
}
# Keep the current value if not present in the mapping
pre_test_data[tes_const.Q4_KEY] = pre_test_data[tes_const.Q4_KEY].map(ANS_MAPPING).fillna(pre_test_data[tes_const.Q4_KEY])
post_test_data[tes_const.Q4_KEY] = post_test_data[tes_const.Q4_KEY].map(ANS_MAPPING).fillna(post_test_data[tes_const.Q4_KEY])

### Correct Q5 columns values (bug in the form)

Change answer "B)Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !", in  "D)Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !"

In [ ]:
old_value = "B)Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !"
new_value = "D)Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !Hey ! Hey ! Hey ! Hey !"

# Count rows matching old value
pre_test_impacted_lines = (pre_test_data[tes_const.Q5_KEY] == old_value).sum()
post_test_impacted_lines = (post_test_data[tes_const.Q5_KEY] == old_value).sum()
print("Pre-test impacted lines :", pre_test_impacted_lines)
print("Post-test impacted lines :", post_test_impacted_lines)

# Apply replacement
pre_test_data[tes_const.Q5_KEY] = pre_test_data[tes_const.Q5_KEY].replace(old_value, new_value)
post_test_data[tes_const.Q5_KEY] = post_test_data[tes_const.Q5_KEY].replace(old_value, new_value)

### Merge the gender columns for group A and group B+C in post-test data

In [ ]:
# Create a mapping from student id to group id
student_to_group_mapping = {}
for student in stu_const.ALL_STUDENTS:
    student_to_group_mapping[student[stu_const.STUDENT_ID]] = student[stu_const.GROUP_ID]

# Function to get the correct gender value depending on the group
def get_gender_value(row):
    student_id = row[tes_const.STUDENT_ID_KEY]
    group_id = student_to_group_mapping.get(student_id, None)

    if group_id is None:
        return None  # Student not kept in experimentation

    if group_id == stu_const.GROUP_A:
        return row[tes_const.QK_KEY]  # gender from QK for group A
    else:
        return row[tes_const.QH_KEY]  # gender from QH for groups B and C

# Apply to post test
post_test_data[tes_const.GENDER_KEY] = post_test_data.apply(get_gender_value, axis=1)
# Delete QK and QH columns from post test
post_test_data = post_test_data.drop(columns=[tes_const.QH_KEY,tes_const.QK_KEY], errors="ignore")


### Normalisation of gender column

Normalization to `M`-`F`-`O`-`NA` (i.e., Male / Female / Other / No Answer)

In [ ]:
# Get all unique values (strip and lower case) -> used to make lists below
print(post_test_data[tes_const.GENDER_KEY].str.strip().str.lower().unique())

In [ ]:
MALE_VALUES = ["homme","h","masculin","home","m","amsculin","masculine","garçon","hgomme"]
FEMALE_VALUES = ["femme","fille","f","feminin","féminin","femelle","je suis une femme","femme ^-^","fame"]

def mapping_function(gender_value):
     # nan / None and empty value
    if pd.isna(gender_value) or str(gender_value).strip() == "":
        return tes_const.GENDER_NO_ANSWER  
    gender_value_clean = str(gender_value).strip().lower()
    if gender_value_clean in MALE_VALUES:
        return tes_const.GENDER_MALE
    elif gender_value_clean in FEMALE_VALUES:
        return tes_const.GENDER_FEMALE
    else:
        return tes_const.GENDER_OTHER
              
post_test_data[tes_const.GENDER_KEY] = post_test_data[tes_const.GENDER_KEY].apply(mapping_function)


In [ ]:
# Checking
print(post_test_data[tes_const.GENDER_KEY].unique())

### Simplifying Multiple-choice question answers in both pre and post-tests

In [ ]:
# Columns to modify
cols_to_trim = tes_const.PROGRAMMING_QUESTIONS
# Keep only the first character of specified columns (i.e. A/B/C/D/E)
pre_test_data[cols_to_trim] = pre_test_data[cols_to_trim].apply(lambda x: x.str[0])
post_test_data[cols_to_trim] = post_test_data[cols_to_trim].apply(lambda x: x.str[0])

In [ ]:
# Checking
print("Pre-test :")
for question in tes_const.PROGRAMMING_QUESTIONS:
    values = sorted(pre_test_data[question].dropna().unique())
    print(f"{question} : {values}")
print("\nPost-test :")
for question in tes_const.PROGRAMMING_QUESTIONS:
    values = sorted(post_test_data[question].dropna().unique())
    print(f"{question} : {values}")

### Deletion of unsubmitted tests (creates outliers)

In [ ]:
print(f"Number of pre-test before delete: {len(pre_test_data)}")
deleted_pre_ids = pre_test_data.loc[pre_test_data[tes_const.T2_KEY].isna(), tes_const.STUDENT_ID_KEY].tolist()
print(f"Deleted pre-test student_ids: {deleted_pre_ids}")
pre_test_data = pre_test_data[pre_test_data[tes_const.T2_KEY].notna()]
print(f"Number of pre-test after delete: {len(pre_test_data)}\n")

print(f"Number of post-test before delete: {len(post_test_data)}")
deleted_post_ids = post_test_data.loc[post_test_data[tes_const.T2_KEY].isna(), tes_const.STUDENT_ID_KEY].tolist()
print(f"Deleted post-test student_ids: {deleted_post_ids}")
post_test_data = post_test_data[post_test_data[tes_const.T2_KEY].notna()]
print(f"Number of post-test after delete: {len(post_test_data)}")


### Correction of student_id errors

Student 381 entered the wrong student_id during his post-test (entered 382)

In [ ]:
mask = (
    post_test_data[tes_const.STUDENT_ID_KEY] == 382
) & (
    post_test_data[tes_const.T2_KEY] == pd.to_datetime("04/12/2025 09:41",format="%d/%m/%Y %H:%M")
)

print("Found matches :", mask.sum())
print(post_test_data.loc[mask, [tes_const.STUDENT_ID_KEY, tes_const.T2_KEY]])

post_test_data.loc[mask, tes_const.STUDENT_ID_KEY] = 381

## 5/ Data filtration


### 5.1 / Interaction traces

### Check students activity outside the sessions

We removed from the data students who progressed through a level outside of the sessions (see commented lines on `src/students_constants.py`). We kept students who performed other actions that did not lead to progression in a level.

In [ ]:
# Dictionary to store first, last timestamps and count per student
students_out_of_session = {}

# Iterate over each class
for class_name, sessions in ses_const.SESSION_DATE.items():
    
    if class_name in stu_const.CLASS_MAPPING:
        class_students = [s[stu_const.GAME_ID] for s in stu_const.CLASS_MAPPING[class_name]]
    else:
        print(f"Class {class_name} not found in class_mapping")
        continue
    
    # Create session intervals
    session_intervals = [
        (pd.to_datetime(s['start'], format="%Y-%m-%d %H:%M:%S.%f"),
         pd.to_datetime(s['end'],   format="%Y-%m-%d %H:%M:%S.%f"))
        for s in sessions.values()
    ]
    
    # Filter traces for students in this class
    class_traces = interaction_data[interaction_data[int_const.GAME_ID_DATA_KEY].isin(class_students)]
    
    # Check each trace
    for idx, row in class_traces.iterrows():
        trace_time = row[int_const.DATE_DATA_KEY]
        trace_student = row[int_const.GAME_ID_DATA_KEY]
        
        # Check if trace_time is outside all sessions
        if not any(start <= trace_time <= end for start, end in session_intervals):
            if trace_student not in students_out_of_session:
                # Initialize: first, last, count
                students_out_of_session[trace_student] = [trace_time, trace_time, 1]
            else:
                # Update first, last and increment count
                students_out_of_session[trace_student][0] = min(students_out_of_session[trace_student][0], trace_time)
                students_out_of_session[trace_student][1] = max(students_out_of_session[trace_student][1], trace_time)
                students_out_of_session[trace_student][2] += 1



print("STUDENTS WITH TRACES OUTSIDE SESSIONS :\n")

if students_out_of_session:
    for game_id, (first_time, last_time, count) in sorted(students_out_of_session.items()):
        print(f"{game_id}: {count} traces out of session - first = {first_time}, last = {last_time}")
else:
    print("No traces found outside sessions!")

print(f"\nTotal students with out-of-session traces: {len(students_out_of_session)}")


###  Filtration of interaction traces on students and sessions dates


In [ ]:
print("Total number of students in the study :", len(stu_const.ALL_STUDENTS))
print("Number of interaction traces initially :", len(interaction_data))

# Convert session dates to datetime
for class_id, sessions in ses_const.SESSION_DATE.items():
    for session_id, bounds in sessions.items():
        bounds["start"] = pd.to_datetime(bounds["start"], format="%Y-%m-%d %H:%M:%S.%f")
        bounds["end"]   = pd.to_datetime(bounds["end"],   format="%Y-%m-%d %H:%M:%S.%f")

# New clean DataFrame for filtered interaction traces
filtered_interaction = []

# Process class by class
for class_id, sessions in ses_const.SESSION_DATE.items():
    # Get students in this class
    class_students = [s[stu_const.GAME_ID] for s in stu_const.CLASS_MAPPING[class_id]]
    # Get session intervals for this class
    class_intervals = [
        (bounds["start"], bounds["end"])
        for bounds in sessions.values()
    ]
    # Extract only traces from these students
    class_traces = interaction_data[
        interaction_data[int_const.GAME_ID_DATA_KEY].isin(class_students)
    ]
    # Filter traces inside this class’s sessions
    class_mask = False
    for start, end in class_intervals:
        class_mask |= class_traces[int_const.DATE_DATA_KEY].between(start, end)

    valid_traces = class_traces.loc[class_mask]
    # Append to global filtered dataset
    filtered_interaction.append(valid_traces)

# Concatenate all filtered traces
interaction_data = pd.concat(filtered_interaction, ignore_index=True)

print("Number of interaction traces after student and date filtration:", len(interaction_data))

# Summary per day
traces_per_day = (
    interaction_data
        .groupby(interaction_data[int_const.DATE_DATA_KEY].dt.date)
        .size()
        .reset_index(name="nb_traces")
        .rename(columns={int_const.DATE_DATA_KEY: "date"})
)

print("Number of interaction traces by days:")
print(traces_per_day)



### 5.2 / Pre and post-tests

###  Filtration of pre and post-tests on students of the experimentation

In [ ]:
# Filter tests to keep only traces from valid students in our study
valid_student_ids = [student[stu_const.STUDENT_ID] for student in stu_const.ALL_STUDENTS]
pre_test_data = pre_test_data[
    pre_test_data[tes_const.STUDENT_ID_KEY].isin(valid_student_ids)
]
post_test_data = post_test_data[
    post_test_data[tes_const.STUDENT_ID_KEY].isin(valid_student_ids)
]
print("Number of students in pre-test :", pre_test_data[tes_const.STUDENT_ID_KEY].nunique())
print("Number of students in post-test:", post_test_data[tes_const.STUDENT_ID_KEY].nunique())

## 6/ Checkings


### Check if students are missing in datasets

In [ ]:
# Extract all expected student ids and game ids
expected_student_ids = [student[stu_const.STUDENT_ID] for student in stu_const.ALL_STUDENTS]
expected_game_ids = [student[stu_const.GAME_ID] for student in stu_const.ALL_STUDENTS]

# Check presence in interaction traces
students_with_interactions = interaction_data[int_const.GAME_ID_DATA_KEY].unique()
missing_interactions = set(expected_game_ids) - set(students_with_interactions)
print("=== Interaction traces check ===")
print("Expected game ids:", len(expected_game_ids))
print("Found in traces:", len(students_with_interactions))
print("Missing game ids:")
if missing_interactions:
    print(sorted(missing_interactions))
else:
    print("No missing")

# Check presence in pre-test data
students_in_pre = pre_test_data[tes_const.STUDENT_ID_KEY].unique()
missing_pre = set(expected_student_ids) - set(students_in_pre)
print("\n=== Pre-test check ===")
print("Expected student ids:", len(expected_student_ids))
print("Found in pre-test:", len(students_in_pre))
print("Missing student ids:")
if missing_pre:
    print(sorted(missing_pre))
else:
    print("No missing")

# Check presence in post-test data
students_in_post = post_test_data[tes_const.STUDENT_ID_KEY].unique()
missing_post = set(expected_student_ids) - set(students_in_post)
print("\n=== Post-test check ===")
print("Expected student ids:", len(expected_student_ids))
print("Found in post-test:", len(students_in_post))
print("Missing student ids:")
if missing_pre:
    print(sorted(missing_post))
else:
    print("No missing")


### Check of submission uniqueness in pre and post-tests

In [ ]:
multi_pre = pre_test_data[tes_const.STUDENT_ID_KEY].value_counts()
multi_pre = multi_pre[multi_pre > 1]

print("Students with multiple submissions in pre-test:")
print(multi_pre.to_string() if not multi_pre.empty else "None")

In [ ]:
multi_post = post_test_data[tes_const.STUDENT_ID_KEY].value_counts()
multi_post = multi_post[multi_post > 1]

print("Students with multiple submissions in post-test:")
print(multi_post.to_string() if not multi_post.empty else "None")

### Check that students are present on each sessions

We removed from data students who were absent from at least one session (see commented lines on `src/students_constants.py`). Except if the student had completed the game and was not active in the next sessions.

In [ ]:
# Dictionary to store students who missed at least one session
students_with_absences = {}

# Analyze attendance for each class
for class_name, sessions in ses_const.SESSION_DATE.items():
    print(f"\n=== {class_name} ===")
    
    # Get student list for this class using the mapping
    if class_name in stu_const.CLASS_MAPPING:
        class_students = [s[stu_const.GAME_ID] for s in stu_const.CLASS_MAPPING[class_name]]
    else:
        print(f"Class {class_name} not found in class_mapping")
        continue
    
    # Check each session
    for session_name, session_times in sessions.items():
        start_time = pd.to_datetime(session_times['start'], format="%Y-%m-%d %H:%M:%S.%f")
        end_time = pd.to_datetime(session_times['end'],  format="%Y-%m-%d %H:%M:%S.%f")
        
        # Filter traces for this session
        session_traces = interaction_data[
            (interaction_data[int_const.DATE_DATA_KEY] >= start_time) & 
            (interaction_data[int_const.DATE_DATA_KEY] <= end_time)
        ]
        
        # Students present (with at least one activity)
        present_students = set()
        for game_id in session_traces[int_const.GAME_ID_DATA_KEY].unique():
            present_students.add(game_id)
        
        # Students absent
        absent_students = set(class_students) - present_students
        
        if absent_students:
            print(f"Session {session_name}: {len(absent_students)} absent(s) - {sorted(absent_students)}")
            
            # Track students with absences
            for game_id in absent_students:
                if game_id not in students_with_absences:
                    students_with_absences[game_id] = []
                students_with_absences[game_id].append(f"{class_name}_{session_name}")
        else:
            print(f"Session {session_name}: All students present")

# Display summary of students with absences
print("\n=== SUMMARY : STUDENT WHO MISSED AT LEAST ONE SESSION ===")

if students_with_absences:
    for game_id, missed_sessions in sorted(students_with_absences.items()):
        print(f"{game_id}: missed {len(missed_sessions)} session(s) - {missed_sessions}")
else:
    print("All students attended all sessions!")

print(f"\nTotal students with absences: {len(students_with_absences)}")

## 7/ Data enriching for futur splitting and filtration

### Interaction data

In [ ]:
# Create a mapping from game id to group id
game_to_group_mapping = {}
for student in stu_const.ALL_STUDENTS:
    game_to_group_mapping[student[stu_const.GAME_ID]] = student[stu_const.GROUP_ID]

# Create a mapping from game id to student id
game_to_student_mapping = {}
for student in stu_const.ALL_STUDENTS:
    game_to_student_mapping[student[stu_const.GAME_ID]] = student[stu_const.STUDENT_ID]

# Add group id and student id columns to the traces dataframe
interaction_data[int_const.GROUP_ID_DATA_KEY] = interaction_data[int_const.GAME_ID_DATA_KEY].map(game_to_group_mapping)
interaction_data[int_const.STUDENT_ID_DATA_KEY] = interaction_data[int_const.GAME_ID_DATA_KEY].map(game_to_student_mapping)

# Count number of unique students per group
students_per_group = interaction_data.groupby(int_const.GROUP_ID_DATA_KEY)[int_const.GAME_ID_DATA_KEY].nunique()
print("\n=== Number of students by group ===")
for group, count in students_per_group.items():
    print(f"Group {group}: {count} students")
print(f"TOTAL: {students_per_group.sum()} students")

# Count traces per group
traces_per_group = interaction_data[int_const.GROUP_ID_DATA_KEY].value_counts().sort_index()
print("\n=== Distribution of traces by group ===")
print(traces_per_group)
print(f"TOTAL: {traces_per_group.sum()} traces")


### Pre and post-tests

In [ ]:
# Create a mapping from student id to group id
student_to_group_mapping = {}
for student in stu_const.ALL_STUDENTS:
    student_to_group_mapping[student[stu_const.STUDENT_ID]] = student[stu_const.GROUP_ID]

# Create a mapping from student id to game id
student_to_game_mapping = {}
for student in stu_const.ALL_STUDENTS:
    student_to_game_mapping[student[stu_const.STUDENT_ID]] = student[stu_const.GAME_ID]

# Add group id and game id columns to the tests dataframe
pre_test_data[tes_const.GROUP_ID_KEY] = pre_test_data[tes_const.STUDENT_ID_KEY].map(student_to_group_mapping)
pre_test_data[tes_const.GAME_ID_KEY] = pre_test_data[tes_const.STUDENT_ID_KEY].map(student_to_game_mapping)
post_test_data[tes_const.GROUP_ID_KEY] = post_test_data[tes_const.STUDENT_ID_KEY].map(student_to_group_mapping)
post_test_data[tes_const.GAME_ID_KEY] = post_test_data[tes_const.STUDENT_ID_KEY].map(student_to_game_mapping)

# Count pre-test per group
pre_test_per_group = pre_test_data[tes_const.GROUP_ID_KEY].value_counts().sort_index()
print("\n=== Distribution of pre-test by group ===")
print(pre_test_per_group)
print(f"TOTAL: {pre_test_per_group.sum()} pre-test")

# Count post-test per group
post_test_per_group = post_test_data[tes_const.GROUP_ID_KEY].value_counts().sort_index()
print("\n=== Distribution of post-test by group ===")
print(post_test_per_group)
print(f"TOTAL: {post_test_per_group.sum()} post-test")


## 8/ Data exportation

In [ ]:
# Export to Excel file for manual checking
interaction_data.to_excel("../data/cleaned/cleaned_data_interaction_traces.xlsx")
pre_test_data.to_excel("../data/cleaned/cleaned_data_pre_test.xlsx")
post_test_data.to_excel("../data/cleaned/cleaned_data_post_test.xlsx")

In [ ]:
# Export du Pickle format for the following notebooks analyses
interaction_data.to_pickle("../data/interim/interaction_data.pkl")
pre_test_data.to_pickle("../data/interim/pre_test_data.pkl")
post_test_data.to_pickle("../data/interim/post_test_data.pkl")